## Machine Learning Supervised

## Table of Contents

So far, we have focused on using KNN as our model to predict California housing prices. However, there are other models worth exploring. Today, we will experiment with both simple Linear Regression and Decision Trees to understand how they explain our target variable. In machine learning, we typically choose our model based on the relationship between our features and the target variable, or simply by selecting the model with the higher score

Yesterday, we applied some feature engineering techniques, and our model indeed increased its performance. Now, let's see how Linear Regression and Decision Tree perform when we apply the same feature engineering techniques.

#### Loading and preparing the data

In [1]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
california = fetch_california_housing()
print(california["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [3]:
df_cali = pd.DataFrame(california["data"], columns = california["feature_names"])
df_cali["median_house_value"] = california["target"]

df_cali.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,median_house_value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


#### Normalization & Feature Selection

Like we did in Feature Engineering lesson, we are going to normalize our data and select a subset of columns as our features.

#### Train Test Split

In [4]:
features = df_cali.drop(columns = ["median_house_value","AveOccup", "Population", "AveBedrms"])
target = df_cali["median_house_value"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

Create an instance of the normalizer

In [6]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [7]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [8]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,MedInc,HouseAge,AveRooms,Latitude,Longitude
0,0.257838,0.098039,0.048751,0.137088,0.677291
1,0.268265,1.000000,0.031762,0.551541,0.190239
2,0.236783,0.490196,0.027097,0.137088,0.632470
3,0.066578,0.725490,0.017987,0.156217,0.606574
4,0.184591,1.000000,0.023207,0.163656,0.596614


In [9]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,MedInc,HouseAge,AveRooms,Latitude,Longitude
0,0.251852,0.411765,0.034147,0.004251,0.727092
1,0.364112,0.607843,0.037296,0.146652,0.635458
2,0.265431,0.549020,0.036045,0.649309,0.250000
3,0.134564,0.705882,0.029397,0.070138,0.871514
4,0.310685,0.470588,0.024621,0.557917,0.191235


## Linear Regression

Let's create an instance of Linear Regression model.

In [10]:
lin_reg = LinearRegression()

Training Linear Regression with our normalized data

In [11]:
lin_reg.fit(X_train_norm, y_train)

LinearRegression()

Evaluate model's performance

In [12]:
pred = lin_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", lin_reg.score(X_test_norm, y_test))

MAE 0.547840183140973
RMSE 0.7424116296739353
R2 score 0.5773051453722333


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression yielding a worse score than our previous model, KNN.

In Linear Regression, we often assess feature importance by examining the coefficients in the model. These coefficients indicate the impact of each feature on the model's predictions.

- Determine the coefficients (β) in the linear regression equation corresponding to each feature.
- The magnitude of these coefficients reflects the relative importance of the features. **Greater absolute values suggest more substantial impacts.**

In [13]:
lin_reg_coef = {feature : coef for feature, coef in zip(X_train_norm.columns, lin_reg.coef_)}
lin_reg_coef

{'MedInc': 5.3846460612069045,
 'HouseAge': 0.49426222880241316,
 'AveRooms': 2.4771072369823175,
 'Latitude': -4.293927324319669,
 'Longitude': -4.660457306574648}

We can conclude that **Median Income** have the highest impact in our model.

## Decision Tree

So far between KNN and Liner Regression, the first yield a better score, let's see how a Decision Tree performs.

- Initialize a Decision Tree instance

- Setting max_depth as 10, this means we will allow our tree to split 10 times

In [14]:
tree = DecisionTreeRegressor(max_depth=10)

- Training the model

In [15]:
tree.fit(X_train_norm, y_train)

DecisionTreeRegressor(max_depth=10)

- Evaluate the model

In [16]:
X_train_norm

,MedInc,HouseAge,AveRooms,Latitude,Longitude
0,0.257838,0.098039,0.048751,0.137088,0.677291
1,0.268265,1.000000,0.031762,0.551541,0.190239
2,0.236783,0.490196,0.027097,0.137088,0.632470
3,0.066578,0.725490,0.017987,0.156217,0.606574
4,0.184591,1.000000,0.023207,0.163656,0.596614
...,...,...,...,...,...
16507,0.269831,0.372549,0.036536,0.608927,0.307769
16508,0.166453,0.509804,0.031923,0.524973,0.344622
16509,0.186053,0.588235,0.019814,0.429330,0.244024
16510,0.353899,0.647059,0.026190,0.114772,0.639442


In [17]:
y_test

14740    1.369
10101    2.413
20566    2.007
2670     0.725
15709    4.600
         ...  
6655     1.695
3505     2.046
1919     1.286
1450     2.595
4148     1.676
Name: median_house_value, Length: 4128, dtype: float64

In [18]:
pred = tree.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", tree.score(X_test_norm, y_test))

MAE 0.4233100425359355
RMSE 0.6279720148376425
R2 score 0.697574777533179


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
#Not much impact if we scale our data. Important for KNN not for regression

Often we check what are the most relevant features, like we did before in Linear Regression.

In [19]:
tree_importance = {feature : importance for feature, importance in zip(X_train_norm.columns, tree.feature_importances_)}
tree_importance           

{'MedInc': 0.6220642589515147,
 'HouseAge': 0.05879191285611778,
 'AveRooms': 0.04566224484380877,
 'Latitude': 0.14035393284854983,
 'Longitude': 0.13312765050000885}

In [20]:
from sklearn.tree import export_text

tree_viz = export_text(tree, feature_names=list(X_train_norm.columns))
print(tree_viz)
#value = Actual real price
#Is MedInc sup to 0.31 Yes/No then next
#Python chooses 0.31 because it creates the clearest separation between house prices, helping the decision tree make better predictions

|--- MedInc <= 0.31
|   |--- MedInc <= 0.18
|   |   |--- AveRooms <= 0.02
|   |   |   |--- MedInc <= 0.12
|   |   |   |   |--- AveRooms <= 0.02
|   |   |   |   |   |--- Latitude <= 0.16
|   |   |   |   |   |   |--- Longitude <= 0.59
|   |   |   |   |   |   |   |--- value: [4.50]
|   |   |   |   |   |   |--- Longitude >  0.59
|   |   |   |   |   |   |   |--- Longitude <= 0.72
|   |   |   |   |   |   |   |   |--- HouseAge <= 0.23
|   |   |   |   |   |   |   |   |   |--- AveRooms <= 0.02
|   |   |   |   |   |   |   |   |   |   |--- value: [3.50]
|   |   |   |   |   |   |   |   |   |--- AveRooms >  0.02
|   |   |   |   |   |   |   |   |   |   |--- value: [3.25]
|   |   |   |   |   |   |   |   |--- HouseAge >  0.23
|   |   |   |   |   |   |   |   |   |--- Latitude <= 0.13
|   |   |   |   |   |   |   |   |   |   |--- value: [1.94]
|   |   |   |   |   |   |   |   |   |--- Latitude >  0.13
|   |   |   |   |   |   |   |   |   |   |--- value: [1.49]
|   |   |   |   |   |   |   |--- Longitude >  

A bit overwhelming to see, let's use graphviz library.

**Note**: you will need to install graphivz - pip install graphviz

- We will train a decision tree, in this case with max_depth=2 to better see the diagram

In [23]:
pip install graphviz

   ---------------------------------------- 0.0/47.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/47.1 kB ? eta -:--:--
   ----------------- ---------------------- 20.5/47.1 kB 320.0 kB/s eta 0:00:01
   ---------------------------------- ----- 41.0/47.1 kB 388.9 kB/s eta 0:00:01
   ---------------------------------------- 47.1/47.1 kB 335.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

tree = DecisionTreeRegressor(max_depth=2)
tree.fit(X_train_norm, y_train)


dot_data = export_graphviz(tree, out_file="tree.dot", filled=True, rounded=True, feature_names=X_train_norm.columns)

with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH